In [1]:
!pip install transformers
!pip install torch

In [2]:
import pandas as pd
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
df_test = pd.read_csv('/content/gdrive/MyDrive/HASOC23/guj_test_translated_combined.csv', delimiter = ',')
df_test

,tweet_id,text,translated,combined
0,2023_test_main_tweet_1,વધારે પડતાં લાગણીશીલ ન બનતા \nઆજના જમાના માં લ...,Not overly emotional\nIn today's time the emot...,વધારે પડતાં લાગણીશીલ ન બનતા \nઆજના જમાના માં લ...
1,2023_test_main_tweet_2,@AUTHOR બસ કર બે ટોપા..,@Author Bus Tax Two Top ..,@AUTHOR બસ કર બે ટોપા.. @Author Bus Tax Two To...
2,2023_test_main_tweet_3,@AUTHOR @USER @USER બાયડી નો ભાઇ આયવો ભડવો 🤣😂,@AUTHOR @user @user Bidi's brother Avo,@AUTHOR @USER @USER બાયડી નો ભાઇ આયવો ભડવો 🤣😂 ...
3,2023_test_main_tweet_4,@AUTHOR ગુજરાત પોલીસ ઈસ તરહ બેવજહ મુસલમાન કો પ...,@Author Gujarat Police Is Tarah Bigaha Muslim ...,@AUTHOR ગુજરાત પોલીસ ઈસ તરહ બેવજહ મુસલમાન કો પ...
4,2023_test_main_tweet_5,@AUTHOR ગુજરાત માં ભરૂચ જિલ્લામાં આવેલો જંબુસર...,@Author in Jambusar taluka in Bharuch district...,@AUTHOR ગુજરાત માં ભરૂચ જિલ્લામાં આવેલો જંબુસર...
...,...,...,...,...
1191,2023_test_main_tweet_1192,"@AUTHOR સાચું કહે છે, પણ નકલી ગાંધી છે\n(૧) નક...","@Author is right, but fake Gandhi\n(2) Fake Ga...","@AUTHOR સાચું કહે છે, પણ નકલી ગાંધી છે\n(૧) નક..."
1192,2023_test_main_tweet_1193,@AUTHOR ચોર છે બધા,@Author is a thief all,@AUTHOR ચોર છે બધા @Author is a thief all
1193,2023_test_main_tweet_1194,@AUTHOR મંદ ચાલશે પણ ચોર નઈ ચાલે,@Author will slow but the thief is not,@AUTHOR મંદ ચાલશે પણ ચોર નઈ ચાલે @Author will ...
1194,2023_test_main_tweet_1195,@AUTHOR આવું વાહિયાત ખાવા વાળા લોકો પણ છે 😈😈😈😈...,@Author is also a fucking people.\n\n@User\n\n...,@AUTHOR આવું વાહિયાત ખાવા વાળા લોકો પણ છે 😈😈😈😈...


In [4]:
'''from sklearn.model_selection import train_test_split
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df_guj['label'] = le.fit_transform(df_guj.label.values)

df_train, df_test = train_test_split(df_guj, test_size=0.15, stratify = df_guj['label'], random_state=42)'''

"from sklearn.model_selection import train_test_split\nfrom sklearn import preprocessing\nle = preprocessing.LabelEncoder()\ndf_guj['label'] = le.fit_transform(df_guj.label.values)\n\ndf_train, df_test = train_test_split(df_guj, test_size=0.15, stratify = df_guj['label'], random_state=42)"

In [5]:
import torch
from transformers import BertModel, BertTokenizerFast


tokenizer = BertTokenizerFast.from_pretrained("setu4993/LaBSE")
if tokenizer.pad_token is None:
  print('adding token')
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [6]:
from transformers import BertModel
from torch import nn
class CustomBERTModel(nn.Module):
    def __init__(self, model):
          super(CustomBERTModel, self).__init__()
          self.bert = model
          ### New layers:
          self.dropout = nn.Dropout(0.3)
          self.linear1 = nn.Linear(768, 256)
          self.relu = nn.LeakyReLU(negative_slope=0.01)
          self.linear2 = nn.Linear(256, 1) ## 2 is the number of classes in this example
          self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
          output = self.bert(
               input_ids,
               attention_mask=attention_mask)
          #print(output)
          pooler_output = self.dropout(output['pooler_output'])
          #print(sequence_output.shape())
          # sequence_output has the following shape: (batch_size, sequence_length, 768)
          linear1_output = self.linear1(pooler_output) ## extract the 1st token's embeddings
          relu_output = self.relu(linear1_output)
          linear2_output = self.linear2(relu_output)
          sigmoid_output = self.sigmoid(linear2_output)

          return sigmoid_output

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [8]:
model = torch.load('/content/gdrive/MyDrive/saved_models/last-LA-BSE-Sigmoid-deep-translated-5ep-guj',map_location=torch.device('cpu'))
model.eval()
model.to(device)

CustomBERTModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(501153, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [9]:
df_test

,tweet_id,text,translated,combined
0,2023_test_main_tweet_1,વધારે પડતાં લાગણીશીલ ન બનતા \nઆજના જમાના માં લ...,Not overly emotional\nIn today's time the emot...,વધારે પડતાં લાગણીશીલ ન બનતા \nઆજના જમાના માં લ...
1,2023_test_main_tweet_2,@AUTHOR બસ કર બે ટોપા..,@Author Bus Tax Two Top ..,@AUTHOR બસ કર બે ટોપા.. @Author Bus Tax Two To...
2,2023_test_main_tweet_3,@AUTHOR @USER @USER બાયડી નો ભાઇ આયવો ભડવો 🤣😂,@AUTHOR @user @user Bidi's brother Avo,@AUTHOR @USER @USER બાયડી નો ભાઇ આયવો ભડવો 🤣😂 ...
3,2023_test_main_tweet_4,@AUTHOR ગુજરાત પોલીસ ઈસ તરહ બેવજહ મુસલમાન કો પ...,@Author Gujarat Police Is Tarah Bigaha Muslim ...,@AUTHOR ગુજરાત પોલીસ ઈસ તરહ બેવજહ મુસલમાન કો પ...
4,2023_test_main_tweet_5,@AUTHOR ગુજરાત માં ભરૂચ જિલ્લામાં આવેલો જંબુસર...,@Author in Jambusar taluka in Bharuch district...,@AUTHOR ગુજરાત માં ભરૂચ જિલ્લામાં આવેલો જંબુસર...
...,...,...,...,...
1191,2023_test_main_tweet_1192,"@AUTHOR સાચું કહે છે, પણ નકલી ગાંધી છે\n(૧) નક...","@Author is right, but fake Gandhi\n(2) Fake Ga...","@AUTHOR સાચું કહે છે, પણ નકલી ગાંધી છે\n(૧) નક..."
1192,2023_test_main_tweet_1193,@AUTHOR ચોર છે બધા,@Author is a thief all,@AUTHOR ચોર છે બધા @Author is a thief all
1193,2023_test_main_tweet_1194,@AUTHOR મંદ ચાલશે પણ ચોર નઈ ચાલે,@Author will slow but the thief is not,@AUTHOR મંદ ચાલશે પણ ચોર નઈ ચાલે @Author will ...
1194,2023_test_main_tweet_1195,@AUTHOR આવું વાહિયાત ખાવા વાળા લોકો પણ છે 😈😈😈😈...,@Author is also a fucking people.\n\n@User\n\n...,@AUTHOR આવું વાહિયાત ખાવા વાળા લોકો પણ છે 😈😈😈😈...


In [10]:
import pandas as pd
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
class sinhalese_data(Dataset):
  def __init__(self, df_sinh, tokenizer):
    self.dataset = df_sinh
    self.data = list(self.dataset.combined)
   # print(self.data)
    #self.labels = list(self.dataset.label)
    self.tokenizer = tokenizer
    self.indexval = list(self.dataset.index)
    #model = BertModel.from_pretrained("setu4993/LaBSE")

  def __len__(self):
    return len(self.data)
  def __getitem__(self, index):
    example = self.data[index]
    indexvalue= self.indexval[index]
    inputs = tokenizer(example, return_tensors="pt", padding='max_length', max_length=512, truncation=True)
    input_ids = inputs['input_ids'].squeeze()
    attention_mask = inputs['attention_mask'].squeeze()
    #print(self.labels[index])
    #labels = self.labels[index]
    return input_ids, attention_mask,indexvalue#, labels

In [11]:
dataset = sinhalese_data(df_test, tokenizer)

In [12]:
import torch
from torch.nn import functional as F

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.BCELoss()
batch_size = 1
num_epochs = 10
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [13]:
count = 0
for batch in data_loader:

        # Move batch tensors to the device
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        index = batch[2][0].item()
        print(index)
        outputs = model(input_ids=input_ids,attention_mask = attention_mask)
        predictions = outputs.tolist()
        #print(predictions)
        for i in range(len(predictions)):
            predictions[i] = round(predictions[i][0])
        print(predictions)
        df_test.loc[index,'predicted_label'] = predictions[0]
        count+=1


df_test


0
[1]
1
[0]
2
[0]
3
[1]
4
[0]
5
[0]
6
[0]
7
[0]
8
[1]
9
[1]
10
[1]
11
[1]
12
[1]
13
[0]
14
[0]
15
[0]
16
[0]
17
[0]
18
[0]
19
[0]
20
[0]
21
[1]
22
[1]
23
[1]
24
[0]
25
[0]
26
[0]
27
[1]
28
[1]
29
[0]
30
[0]
31
[1]
32
[0]
33
[1]
34
[1]
35
[1]
36
[1]
37
[1]
38
[0]
39
[1]
40
[0]
41
[0]
42
[1]
43
[1]
44
[1]
45
[0]
46
[1]
47
[1]
48
[0]
49
[0]
50
[0]
51
[1]
52
[1]
53
[0]
54
[1]
55
[1]
56
[1]
57
[0]
58
[0]
59
[0]
60
[0]
61
[1]
62
[0]
63
[0]
64
[0]
65
[0]
66
[0]
67
[0]
68
[1]
69
[0]
70
[1]
71
[1]
72
[0]
73
[1]
74
[1]
75
[0]
76
[1]
77
[0]
78
[1]
79
[1]
80
[1]
81
[1]
82
[0]
83
[0]
84
[1]
85
[1]
86
[0]
87
[1]
88
[1]
89
[1]
90
[1]
91
[1]
92
[1]
93
[1]
94
[1]
95
[1]
96
[0]
97
[1]
98
[1]
99
[1]
100
[1]
101
[0]
102
[1]
103
[1]
104
[0]
105
[0]
106
[0]
107
[1]
108
[1]
109
[0]
110
[0]
111
[1]
112
[0]
113
[1]
114
[1]
115
[0]
116
[1]
117
[0]
118
[0]
119
[0]
120
[1]
121
[1]
122
[1]
123
[1]
124
[1]
125
[1]
126
[0]
127
[1]
128
[0]
129
[0]
130
[0]
131
[1]
132
[1]
133
[1]
134
[1]
135
[0]
136
[0]
137
[1]
138
[1

,tweet_id,text,translated,combined,predicted_label
0,2023_test_main_tweet_1,વધારે પડતાં લાગણીશીલ ન બનતા \nઆજના જમાના માં લ...,Not overly emotional\nIn today's time the emot...,વધારે પડતાં લાગણીશીલ ન બનતા \nઆજના જમાના માં લ...,1.0
1,2023_test_main_tweet_2,@AUTHOR બસ કર બે ટોપા..,@Author Bus Tax Two Top ..,@AUTHOR બસ કર બે ટોપા.. @Author Bus Tax Two To...,0.0
2,2023_test_main_tweet_3,@AUTHOR @USER @USER બાયડી નો ભાઇ આયવો ભડવો 🤣😂,@AUTHOR @user @user Bidi's brother Avo,@AUTHOR @USER @USER બાયડી નો ભાઇ આયવો ભડવો 🤣😂 ...,0.0
3,2023_test_main_tweet_4,@AUTHOR ગુજરાત પોલીસ ઈસ તરહ બેવજહ મુસલમાન કો પ...,@Author Gujarat Police Is Tarah Bigaha Muslim ...,@AUTHOR ગુજરાત પોલીસ ઈસ તરહ બેવજહ મુસલમાન કો પ...,1.0
4,2023_test_main_tweet_5,@AUTHOR ગુજરાત માં ભરૂચ જિલ્લામાં આવેલો જંબુસર...,@Author in Jambusar taluka in Bharuch district...,@AUTHOR ગુજરાત માં ભરૂચ જિલ્લામાં આવેલો જંબુસર...,0.0
...,...,...,...,...,...
1191,2023_test_main_tweet_1192,"@AUTHOR સાચું કહે છે, પણ નકલી ગાંધી છે\n(૧) નક...","@Author is right, but fake Gandhi\n(2) Fake Ga...","@AUTHOR સાચું કહે છે, પણ નકલી ગાંધી છે\n(૧) નક...",1.0
1192,2023_test_main_tweet_1193,@AUTHOR ચોર છે બધા,@Author is a thief all,@AUTHOR ચોર છે બધા @Author is a thief all,0.0
1193,2023_test_main_tweet_1194,@AUTHOR મંદ ચાલશે પણ ચોર નઈ ચાલે,@Author will slow but the thief is not,@AUTHOR મંદ ચાલશે પણ ચોર નઈ ચાલે @Author will ...,1.0
1194,2023_test_main_tweet_1195,@AUTHOR આવું વાહિયાત ખાવા વાળા લોકો પણ છે 😈😈😈😈...,@Author is also a fucking people.\n\n@User\n\n...,@AUTHOR આવું વાહિયાત ખાવા વાળા લોકો પણ છે 😈😈😈😈...,1.0


In [14]:
df_test.to_csv('/content/gdrive/MyDrive/HASOC23/Final Results for Gujarati.csv', index=False)